In [1]:
import osmnx as ox
from leuvenmapmatching.map.inmem import InMemMap
import networkx as nx
import geopandas as gpd

In [3]:
#node = gpd.read_file('MOCT_NODE.shp')
link = gpd.read_file('MOCT_LINK.shp')

#노드자료 좌표계 변경 korea2000 -> wgs84
#node = node.to_crs({'init': 'epsg:4326'})

#링크자료 좌표계 변경 korea2000 -> wgs84
link = link.to_crs({'init': 'epsg:4326'})

map 생성

In [4]:
map_con = InMemMap("mymap", use_latlon=True, use_rtree=True, index_edges=True)

In [ ]:
'''
#index를 node_id로 지정
node = node.set_index('NODE_ID')

#node 추가
for nid, row in node[['geometry']].iterrows():
    map_con.add_node(int(nid), (row[0].x, row[0].y))
    
#value error handling: 링크의 시, 종점 노드가 노드 데이터에 없어 발생하는 현상, 없는 노드를 다시 추가해줌
def value_error_handling(f_node, t_node, geom):
    f_node = int(f_node)
    t_node = int(t_node)
    nodes_num = 0
    for coord in geom.coords:
        nodes_num = nodes_num+1
        
    #linestring의 첫 번째는 F_NODE
    map_con.add_node(f_node, (geom.coords[0][0], geom.coords[0][1]))
    
    #linestring의 마지막은 T_NODE
    map_con.add_node(t_node, (geom.coords[nodes_num-1][0], geom.coords[nodes_num-1][1]))
    
    #add_edge
    map_con.add_edge(f_node, t_node)
    
#링크 추가
for nid, row in link[['F_NODE', 'T_NODE', 'geometry']].iterrows():
    try:
        map_con.add_edge(int(row['F_NODE']), int(row['T_NODE']))
    except ValueError:
        value_error_handling(row['F_NODE'], row['T_NODE'], row['geometry'])
'''

In [10]:
for nid, row in link[['F_NODE', 'T_NODE', 'geometry']].iterrows():
    f_node = int(row['F_NODE'])
    t_node = int(row['T_NODE'])
    geom = row['geometry']
    
    nodes_num = 0
    for coord in geom.coords:
        nodes_num = nodes_num+1
    
    #add_node: f_node(시점), t_node(종점)
    map_con.add_node(f_node, (geom.coords[0][1], geom.coords[0][0]))
    map_con.add_node(t_node, (geom.coords[nodes_num-1][1], geom.coords[nodes_num-1][0]))
    
    #add_edge
    map_con.add_edge(f_node, t_node)

gps 데이터 가져오기

In [6]:
import pandas as pd
from rdp import rdp

In [7]:
#gps data 불러오기
gps_data = pd.read_csv('Task1_20191105.csv')

#gps데이터에서 생성시간, 위도, 경도 데이터만 나타내기
gps_data = gps_data[['003_vmcu real vehicle speed','created_on', 'latitude', 'longitude']]
gps_data.rename(columns = {'003_vmcu real vehicle speed' : 'speed'}, inplace = True)

#이상치 제거 - 위도, 경도가 0이고, 속도가 0인 것(정지상태)
gps_data = gps_data[gps_data.latitude !=0]
gps_data = gps_data[gps_data.longitude !=0]
gps_data = gps_data[gps_data.speed !="0.0mph"]

#Florian Wilhelm - Handling GPS Data with Python https://youtu.be/9Q8nEA_0ccg
route = rdp(gps_data[['latitude','longitude']].values, epsilon=1e-5)

In [8]:
#route(array 형식)를 list 형식으로 바꾸기
route_list = [tuple(x) for x in route.tolist()]

leuven map matching 적용하기

In [11]:
from leuvenmapmatching.matcher.distance import DistanceMatcher

In [27]:
matcher = DistanceMatcher(map_con, max_dist=1500, max_dist_init=1500, min_prob_norm=0.1,
                              obs_noise=30, obs_noise_ne=30, dist_noise=10,
                         non_emitting_states = False)
states, _ = matcher.match(route_list)
nodes = matcher.path_pred_onlynodes

In [28]:
nodes

[]

In [21]:
states

[]